In [5]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_curve, classification_report

## Defining the bucket 
s3 = boto3.resource('s3')
bucket_name = 'data-448'
bucket = s3.Bucket(bucket_name)

## Defining the csv files
file_key_train = 'Hwk_Assignments/train.csv'
file_key_test = 'Hwk_Assignments/test.csv'

bucket_object_train = bucket.Object(file_key_train)
file_object_train = bucket_object_train.get()
file_content_stream_train = file_object_train.get('Body')

bucket_object_test = bucket.Object(file_key_test)
file_object_test = bucket_object_test.get()
file_content_stream_test = file_object_test.get('Body')

## Reading the train & test file
train = pd.read_csv(file_content_stream_train, sep = '|')
test = pd.read_csv(file_content_stream_test, sep = '|')

## Reporting the number of observations in each of the data-frames
print('The number of observations in the train data-frame is ', train.shape[0])
print('The number of observations in the test data-frame is ', test.shape[0])

## Frequency table of fraud 
print('The frequency table of fraud is \n', train['fraud'].value_counts() /  train.shape[0])

The number of observations in the train data-frame is  1879
The number of observations in the test data-frame is  498121
The frequency table of fraud is 
 0    0.944651
1    0.055349
Name: fraud, dtype: float64


In [ ]:
## Reporting the number of observations in each of the data-frames